In [1]:
from __future__ import absolute_import, division
from __future__ import print_function, unicode_literals
import pints.toy as toy
import pints
import numpy as np
import logging
import math
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from numpy import inf
import copy 
import pickle
import time
import CMA as CMA

In [ ]:
def CMA_on_single_var(opt, 
                 problem,
                 sampler,
                 sampler_x0, 
                 n_chains=1,
                 chain_size=100,
                 need_sensitivities=False, 
                eval_fun=['ESS']):
    
    optimizer_best_fxs = []
    optimizer_best_xs = []
    
    optimizer_best_fx = np.inf
    optimizer_best_x = 0
    
    for j in range(50):
        # Getting the samples of hyper-parameters for the samplers
        optimizer_xs = opt.ask()
        
        # Saving the score of each sample of hyper-parameters
        optimizer_fxs = []
        print(j,"start samplers run")
        # Evaluate performance for each hyper-parameter configuration
        for x in optimizer_xs:
            
            our_x = [[x[0], x[1], x[2]], x[3], x[4], x[5]]

            # Initialise function evaluations and matrix for chains
            function_evaluations = 0 
            chains = []
            
            # Return array of samples for each chain
            for i in range(n_chains):

                # Create sampler object and set hyperparameter
                curr_x0 = sampler_x0[i]
                mcmc = sampler(curr_x0)
                mcmc.set_hyper_parameters(our_x)
                i_chain = []
                
                # Update until we have ``chain_size`` samples
                # Update function evaluations each time we use ask(),tell()
                while len(i_chain) < chain_size:
                    x = mcmc.ask()
                    if need_sensitivities:
                        fx, grad = problem.evaluateS1(x)
                        sample = mcmc.tell((fx, grad))
                    else:
                        fx,_ = problem.evaluateS1(x)
                        sample = mcmc.tell(fx)
                    function_evaluations += 1
                    if sample is not None:
                        i_chain.append(sample)
                
                # Append ith chain to list of chains       
                chains.append(i_chain)
                
            chains = np.array(chains, copy=True)
            optimizer_fx = 0
            # Calculate the score of the sampler with the given hyper-parameters
            # Get the KL if requested
            if 'KL' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                optimizer_fx = kl/len(chains)                
            
            elif 'KL-ITER' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                avg_kl = kl/len(chains)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = avg_kl * avg_iteration_count
            
            # Get the ESS if requested
            elif 'ESS' in eval_fun:
                ess = np.zeros(chains[0].shape[1])
                for chain in chains:
                    ess += np.array(pints._diagnostics.effective_sample_size(chain))
                print(ess)
                ess /= len(chains)
                ess = np.min(ess)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = math.sqrt(avg_iteration_count) / ess
                
            optimizer_fxs.append(optimizer_fx)
        print(j,"end samplers run")
        opt.tell(optimizer_fxs)
        print(j,"end tell ")
        optimizer_best_fxs.append(opt.fbest())
        optimizer_best_xs.append(opt.xbest())
        
        if opt.fbest() < optimizer_best_fx:
            optimizer_best_fx = opt.fbest()
            optimizer_best_x = opt.xbest()
    
    print(optimizer_best_fx)
    print(optimizer_best_x)
    print()
    
    return optimizer_best_fxs, optimizer_best_xs

In [ ]:
rosenbrock = pints.toy.RosenbrockLogPDF()
eggshape = pints.toy.SimpleEggBoxLogPDF(2,4)
Logistic = pints.toy.LogisticModel()

problem = rosenbrock

# Define the starting points of the samplers
sampler_rosenbrock_x0 = [
    [0,0],
    [3,8],
    [2,4],
]

sampler_egg_x0 = [
    [-10,-10],
    [0,0],
    [10,10],
]

sampler_x0 = sampler_rosenbrock_x0

# Define the starting point of the optimzer
dummy = pints.SliceStepoutMCMC([2, 4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], 
                    dummy.expansion_steps(), dummy.prob_overrelaxed(), dummy.bisection_steps()]
boundary = pints.RectangularBoundaries([0,0,1,0,0],[20,20,100,1,100])

sampler = pints.SliceStepoutMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_single_var(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

In [2]:
dummy = pints.SliceStepoutMCMC([2, 4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], 
                    dummy.expansion_steps(), dummy.prob_overrelaxed(), dummy.bisection_steps()]
print(optimizer_x0)

[0.2, 0.4, 50, 0, 10]


## Results for the Egg Problem

In [ ]:
0.9179869280115049
[ 3.61346418  2.78710331 45.84290531  0.85652318  0.5264789 ]

0.9324991550798959
[ 3.46463821  5.25609066 67.12025263  0.44649369  0.25899178]

0.915696456256111
[ 4.22828699  3.2918315  48.34857415  0.66048817  0.60508832]

0.938988817824792
[ 7.16367172  4.72366301 44.78011937  0.11087744  1.93261219]

0.9176600677810929
[ 3.91173313  3.28633446 55.26513198  0.75707419  0.26133818]

## Results for the Rosenbrock problem

In [ ]:
2.3235648912944455
[17.48272198 16.77697123 59.29593527  0.29438362  3.94942295]

2.16825776726139
[ 7.72599655 14.37161775 62.32867035  0.36297407  0.7233022 ]

1.866527310975345
[ 5.10531175  3.71615054 23.51462788  0.78952023  1.48273688]

2.2561228708422587
[3.38122713e+00 1.49792117e+01 8.08315852e+01 6.64676628e-02
 4.32494415e+00]

1.8751304752745437
[18.06458699  9.45095504 65.12849812  0.51251162  0.716842  ]

## Logistic Problem

In [ ]:
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

class ExampleModel(pints.ForwardModel):
    
    def __init__(self):
        self.model = toy.LogisticModel()
    def simulate(self, x, times):
        return self.model.simulate([x[0]/1000, x[1]], times)
    def simulateS1(self, x, times):
        values, gradient = self.model.simulateS1([x[0]/1000, x[1]], times)
        return values, gradient
    def n_parameters(self):
        return 2

# Then create an instance of our new model class
model = ExampleModel()

In [ ]:
# Create some toy data
real_parameters = [0.015*10000, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.GaussianLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
log_prior = pints.UniformLogPrior(
    [0.01*10000, 400, noise * 0.1],
    [0.02*10000, 600, noise * 100],
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
num_chains = 3
sampler_logistic_x0 = [
    real_parameters * 1.01,
    real_parameters * 0.99,
    real_parameters * 1.01
]

In [ ]:
problem = log_posterior

sampler_x0 = sampler_logistic_x0

# Define the starting point of the optimzer
dummy = pints.SliceStepoutMCMC([2, 4, 10])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.width()[2], 
                dummy.expansion_steps(), dummy.prob_overrelaxed(), dummy.bisection_steps()]

boundary = pints.RectangularBoundaries([0,0,0,1,0,0],[7,7,7,100,1,100])

sampler = pints.SliceStepoutMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_single_var(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

In [3]:
dummy = pints.SliceStepoutMCMC([2, 4, 10])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.width()[2], 
                dummy.expansion_steps(), dummy.prob_overrelaxed(), dummy.bisection_steps()]
print(optimizer_x0)

[0.2, 0.4, 1.0, 50, 0, 10]
